In [320]:
# import library
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset

from sklearn.metrics import accuracy_score

from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers.models.t5.modeling_t5 import T5LayerFF

from datasets import load_dataset

##########################################################################################

import torch
from transformers import (
    BertTokenizer,
    BertForMaskedLM,
)
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

!unzip -qn /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [321]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32

# Load Finetuned Model From Hugging Face

In [322]:
from huggingface_hub import hf_hub_download


tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
bert_model = BertForMaskedLM.from_pretrained("emilyalsentzer/Bio_ClinicalBERT").to(device)

# load the trained model from huggingface
repo_id = "alibababeig/nlp-hw4"
filename = "BioClinicalBert-MLM-Finetuned-40k-25epoch-exp-25epoch-questions.pth"
checkpoint_file = hf_hub_download(repo_id=repo_id, filename=filename)

checkpoint = torch.load(checkpoint_file)
bert_model.load_state_dict(checkpoint["model_state_dict"])
bert_model = bert_model.bert  # dropping MLM head
bert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

# Load Dataset From Hugging Face

In [323]:
from huggingface_hub import hf_hub_download


repo_id = "alibababeig/nlp-hw4"
filename = "MEDMCQA-dataset-with-CLS-40k-25epoch-exp-25epoch-questions-nltk.json"
dataset_path = hf_hub_download(repo_id=repo_id, filename=filename)
loaded_df = pd.read_json(dataset_path)
display(loaded_df)

,question,exp,question_cls
0,"All of the following are pyrogenic cytokines, ...",Interleukin 18 is not a pyrogenic cytokine. IL...,"[0.549001753330231, -0.12343280017375902, 0.15..."
1,40-year old female presented with neck swellin...,Ref. Robbins Pathology. 9th edition. Page. 109...,"[-0.24133916199207303, 0.097042627632618, -0.2..."
2,Following statement regarding dislocation of t...,Anterior dislocation is more common in which h...,"[-0.386974930763245, -0.131634533405304, 0.241..."
3,The active search for unrecognized disease or ...,Screening is the search for unrecognized disea...,"[-0.091413952410221, -0.04559937492013, -0.082..."
4,Fir tree pattern lesion is seen in,Fir tree pattern of distribution of lesions is...,"[-0.41899171471595803, -0.24402141571044902, -..."
...,...,...,...
16826,Carcinoma sigmoid colon with obstruction Manag...,- Obstruction due to rectosigmoid growth with ...,"[0.241796687245369, 0.7256665825843811, -0.431..."
16827,ADHD in childhood can lead to which of the fol...,"ADHD can lead to substance abuse,mood disorder...","[0.20766235888004303, -0.30834984779357905, 0...."
16828,Nerve for adductor compament of thigh ?,Ans. B) Obturator nerveObturator nerve is the ...,"[0.134738609194756, 0.008813104592264, -0.2542..."
16829,The &;a&;wave of jugular venous pulse is produ...,JVP or jugular venous is a reflection of the r...,"[-0.035813611000776, 0.18877704441547402, -0.1..."


In [324]:
print(loaded_df.iloc[1]['question'])
print()
print(loaded_df.iloc[1]['exp'])

40-year old female presented with neck swelling. Gross and histology is shown below.  What is your diagnosis?

Ref. Robbins Pathology. 9th edition. Page. 1099
Medullary carcinoma thyroid
Gross

Single or multiple
Typically nonencapsulated
Solid, gray / tan / yellow, firm, may be infiltrative

Microscopy

Round, polygonal or spindle cells in nests, cords or follicles, defined by sharply outlined fibrous bands
Tumor cells have granular cytoplasm and uniform round / oval nuclei with punctate chromatin
Stroma has amyloid deposits from calcitonin, prominent vascularity with glomeruloid configuration or long cords of vessels, coarse calcifications

 
IHC – Calcitonin


In [325]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)


def encode_text(text, tokenizer, bert_model, max_length=128):
    text = preprocess_text(text)
    tokens = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        outputs = bert_model(**tokens)

    if "pooler_output" in outputs:
        cls_embedding = outputs.pooler_output
    elif "last_hidden_state" in outputs:
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    else:
        raise Exception("No CLS Token found in the given model")
        

    return cls_embedding.cpu()

In [326]:
def cosine_similarity(query, dataset):
    query_norm = query / np.linalg.norm(query)
    dataset_norm = dataset / np.linalg.norm(dataset, axis=1)[:, np.newaxis]
    similarities = np.dot(dataset_norm, query_norm)
    return similarities


def MSE_similarity(query, dataset):
    dists = ((dataset - query) ** 2).sum(axis=1)
    return 1.0 / dists  # inverse of distance scores are equivalent to similarity


def k_nearest_embeddings(query, dataset, k, similarity_metric=cosine_similarity):
    similarities = similarity_metric(query, dataset)

    # Get the indices of the top k highest similarities
    nearest_indices = np.argpartition(similarities, -k)[-k:]

    # Sort these indices by the actual similarities
    nearest_indices = nearest_indices[np.argsort(similarities[nearest_indices])[::-1]]

    # Get the top k similarities and corresponding embeddings
    top_k_similarities = similarities[nearest_indices]
    top_k_embeddings = dataset[nearest_indices]

    return nearest_indices, top_k_embeddings, top_k_similarities


k = 3
# query = "pyrogenic cytokines"
query = "female with neck swelling. Gross and histology."
cls_emb = encode_text(query, tokenizer, bert_model).numpy().squeeze()

In [327]:
nearest_indices, _, nearest_similarities = k_nearest_embeddings(
    cls_emb,
    np.asarray(loaded_df["question_cls"].tolist()),
    k,
    similarity_metric=MSE_similarity,
)
print("Row indices of the k nearest embeddings:", nearest_indices)
print("MSE similarities of the k nearest embeddings:", nearest_similarities)
mins_mse = loaded_df.iloc[nearest_indices]
mins_mse.reset_index(drop=True, inplace=True)
display(mins_mse)

Row indices of the k nearest embeddings: [   1 3456 9274]
MSE similarities of the k nearest embeddings: [0.01963839 0.01770742 0.01723149]


,question,exp,question_cls
0,40-year old female presented with neck swellin...,Ref. Robbins Pathology. 9th edition. Page. 109...,"[-0.24133916199207303, 0.097042627632618, -0.2..."
1,Max Joseph&;s space is a histopathological fea...,Max Joseph's space is a characteristic histolo...,"[-0.09414966404438, -0.05790701508522, 0.25478..."
2,'Mickey Mouse Ears' is a histological feature of:,Paracoccidioidomycosis is a deep fungal infect...,"[0.18600422143936202, -0.17785388231277502, 0...."


In [328]:
nearest_indices, _, nearest_similarities = k_nearest_embeddings(
    cls_emb,
    np.asarray(loaded_df["question_cls"].tolist()),
    k,
    similarity_metric=cosine_similarity,
)
print("Row indices of the k nearest embeddings:", nearest_indices)
print("Cosine similarities of the k nearest embeddings:", nearest_similarities)
mins_cosine = loaded_df.iloc[nearest_indices]
mins_cosine.reset_index(drop=True, inplace=True)
display(mins_cosine)

Row indices of the k nearest embeddings: [   1 3456 7323]
Cosine similarities of the k nearest embeddings: [0.86631365 0.84796207 0.84498733]


,question,exp,question_cls
0,40-year old female presented with neck swellin...,Ref. Robbins Pathology. 9th edition. Page. 109...,"[-0.24133916199207303, 0.097042627632618, -0.2..."
1,Max Joseph&;s space is a histopathological fea...,Max Joseph's space is a characteristic histolo...,"[-0.09414966404438, -0.05790701508522, 0.25478..."
2,A female patient presents with deep Desmoid tu...,Desmoid tumour is a tumour arising from the mu...,"[0.13216152787208602, 0.38578277826309204, -0...."


In [329]:
idx = 0
print(mins_cosine["question"][idx])
print(mins_cosine["exp"][idx])

40-year old female presented with neck swelling. Gross and histology is shown below.  What is your diagnosis?
Ref. Robbins Pathology. 9th edition. Page. 1099
Medullary carcinoma thyroid
Gross

Single or multiple
Typically nonencapsulated
Solid, gray / tan / yellow, firm, may be infiltrative

Microscopy

Round, polygonal or spindle cells in nests, cords or follicles, defined by sharply outlined fibrous bands
Tumor cells have granular cytoplasm and uniform round / oval nuclei with punctate chromatin
Stroma has amyloid deposits from calcitonin, prominent vascularity with glomeruloid configuration or long cords of vessels, coarse calcifications

 
IHC – Calcitonin


# T5 with Fine-tuning

In [330]:
# define configuration
SMALL_MODEL_NAME = 't5-small'
BOTTLENECK_SIZE = 8

In [331]:
# load the model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(SMALL_MODEL_NAME)
finetuned_t5_model = T5ForConditionalGeneration.from_pretrained(SMALL_MODEL_NAME)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [332]:
# adapter layer
class AdapterLayer(nn.Module):
    def __init__(self, emb_dim: int, bottleneck_size: int):

        super().__init__()

        self.sharif_llm_adapter = nn.Sequential(
            nn.Linear(emb_dim, bottleneck_size),
            nn.ReLU(),
            nn.Linear(bottleneck_size, emb_dim)
        )

    def forward(self, x: torch.Tensor):
        adapter_output = self.sharif_llm_adapter(x)
        output =  x + adapter_output
        return output

class FeedForwardAdapterWrapper(nn.Module):
    def __init__(self, original_module: T5LayerFF, bottleneck_size: int):

        super().__init__()
        assert isinstance(original_module, T5LayerFF)

        self.original_module = original_module
        emb_dim = original_module.DenseReluDense.wi.in_features
        self.adapter = AdapterLayer(emb_dim, bottleneck_size)

    def forward(self, x: torch.Tensor):
        output = self.original_module(x)
        output = self.adapter(output)
        return output

In [333]:
def mutate_model_recursive(model: nn.Module, bottleneck_size: int):
    for name, module in model.named_children():
        if isinstance(module, T5LayerFF):
            feed_forward_with_adapter = FeedForwardAdapterWrapper(module, bottleneck_size)
            setattr(model, name, feed_forward_with_adapter)
            print(f"Replaced {name} with FeedForwardAdapterWrapper layer.")
        else:
            mutate_model_recursive(module, bottleneck_size)

def mutate_model(model: nn.Module, bottleneck_size: int):
    if hasattr(model, '_mutated'):
        print("Model already contains adapter layers! \n Try reloading the model.")
        return

    mutate_model_recursive(model, bottleneck_size)

    model._mutated = True
    
mutate_model(finetuned_t5_model, bottleneck_size=BOTTLENECK_SIZE)

Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.


In [334]:
# load the trained model from huggingface
repo_id = "alibababeig/nlp-hw4"
filename = "t5_adapter_weights.pth"
checkpoint_file = hf_hub_download(repo_id=repo_id, filename=filename)

finetuned_t5_model.load_state_dict(torch.load(checkpoint_file))
finetuned_t5_model = finetuned_t5_model.to(device)

In [335]:
# code to generate answer based on model
def generate_answer(row, model):
    model.eval()
    input_text = f"context:{row['exp']}\n question: {row['question']}\n options: 0:{row['opa']}, 1:{row['opb']}, 2:{row['opc']}, 3:{row['opd']}"
    input_ids = tokenizer(input_text, truncation=True, max_length=1024)
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids.to(device), max_length=5)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [336]:
t = {
    'id': '71ed60b6-c4cc-4fad-9c58-1ae876a59842',
    'question': 'Which of the following is most characteristic of diabetic neuropathy?',
    'opa': 'it is usually bilateral',
    'opb': 'pain is not a feature',
    'opc': 'it most commonly affects the brain',
    'opd': 'it spares the autonomic system',
    'cop': 0,
    'choice_type': 'multi',
#     'exp': "Diabetic neuropathy usually presents as peripheral polyneuropathy, usually bilateral, including symptoms of numbness, paresthesia, severe hyperesthesia, and pain. Impairment of proprioceptive fibers can lead to gait abnormalities and Charcot's joints. Mononeuropathy is less common and is often spontaneously reversible. Common syndromes include wrist or foot drop and third, fourth, or sixth cranial nerve palsies. Autonomic neuropathy may cause gastroesophageal dysfunction, bladder dysfunction, and orthostatic hypotension.",
    'subject_name': 'Medicine',
    'topic_name': 'Endocrinology'
}

In [337]:
k = 1

cls_emb = encode_text(t['question'], tokenizer, bert_model).numpy().squeeze()

nearest_indices, _, nearest_similarities = k_nearest_embeddings(
    cls_emb,
    np.asarray(loaded_df["question_cls"].tolist()),
    k,
    similarity_metric=cosine_similarity,
)
print("Row indices of the k nearest embeddings:", nearest_indices)
print("Cosine similarities of the k nearest embeddings:", nearest_similarities)
mins_cosine = loaded_df.iloc[nearest_indices]
mins_cosine.reset_index(drop=True, inplace=True)
display(mins_cosine)

exps = mins_cosine['exp'].tolist()
# exps_str = '\n'.join([f'{i+1}. {exp}' for i, exp in enumerate(exps)])
exps_str = exps[0]
print(exps_str)

t['exp'] = exps_str

Row indices of the k nearest embeddings: [10084]
Cosine similarities of the k nearest embeddings: [0.61111766]


,question,exp,question_cls
0,who totld the dreams are royal road to unconsious,CONTRIBUTIONS OF FREUD o Father of psychoanaly...,"[0.020891854539514, -0.12162970006465901, -0.0..."


CONTRIBUTIONS OF FREUD o Father of psychoanalysis He founded a type of psychotherapy called psychoanalysis. It is nothing but analyzing the psych(MIND) o Interpretation of dreams According to Freud dreams are royal road to unconscious In dreams several conflicts that are present in the unconscious comes to the consciousness in the form of dreams Thus by analyzing ones dreams we could understand the unconscious conflicts o Psychosexual stages of life o Freud divided development into 5 stages namely psycho sexual development o It is divivded namely oral , anal , phallic, latent, genital phase. o Conversion disorders conversion disorder is conveing a psychological pain to physical symptoms present physical symptoms which has some connection with unconscious conflict is called SYMBOLIZATION present physical symptoms which has some resemblance with illness in family members, which is called MODELLING patient has illness like neurological deficit but they have apparent in concern towards the

In [338]:
answer = generate_answer(t, finetuned_t5_model)
print(f"Answer: {answer}")
print(t['cop'])

Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors


Answer: 3
0


# T5 without Fine-tuning

In [346]:
t5_model = T5ForConditionalGeneration.from_pretrained(SMALL_MODEL_NAME)
mutate_model(t5_model, bottleneck_size=BOTTLENECK_SIZE)
t5_model = t5_model.to(device)

Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 1 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.
Replaced 2 with FeedForwardAdapterWrapper layer.


In [347]:
answer = generate_answer(t, t5_model)
print(f"Answer: {answer}")
print(t['cop'])

Answer: o o
0
